# Model using Neural Network

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [2]:
df = pd.read_csv('cleaned_data_v2.csv')

## Check the Columns' Data Type

In [3]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'bmi', 'fitness_goal'],
      dtype='object')

In [6]:
from sklearn.utils import shuffle


shuffled_df = shuffle(df, random_state = 60)

## Split Data for Training and Validation


In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state = 42)

## The training and validation set size

In [8]:
print('training : ', train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181



## Function Transform Dataframe into Tensor Dataset

In [9]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds


## Transform Training and Validation Dataframe into Tensor Dataset

In [10]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [11]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [12]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [13]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [14]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')
bmi          = keras.Input(shape = (1,), name = 'bmi')

In [15]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight, bmi ]

## Encode columns based on their data type

In [16]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)
bmi_encoded = encode_numerical_feature(bmi, 'bmi', train_ds)

In [17]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded, bmi_encoded])

## Build the model architecture

In [18]:
x = keras.layers.Dense(30, activation = 'relu')(all_features)
x = keras.layers.Dense(20, activation = 'relu')(x)
x = keras.layers.Dense(10, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [19]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [20]:
from IPython.core.display import display, HTML

# Enable horizontal scrolling for output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Generate a long output
long_output = "A" * 1000

model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 6s 32ms/step - loss: 0.7176 - accuracy: 0.5187 - val_loss: 0.6990 - val_accuracy: 0.4807
Epoch 2/300
37/37 [==============================] - 0s 8ms/step - loss: 0.7045 - accuracy: 0.5118 - val_loss: 0.6998 - val_accuracy: 0.4530
Epoch 3/300
37/37 [==============================] - 0s 7ms/step - loss: 0.7083 - accuracy: 0.4841 - val_loss: 0.6986 - val_accuracy: 0.4807
Epoch 4/300
37/37 [==============================] - 1s 16ms/step - loss: 0.6900 - accuracy: 0.5477 - val_loss: 0.6990 - val_accuracy: 0.4696
Epoch 5/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6912 - accuracy: 0.5131 - val_loss: 0.6976 - val_accuracy: 0.5304
Epoch 6/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.5353 - val_loss: 0.7014 - val_accuracy: 0.4696
Epoch 7/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6846 - accuracy: 0.5450 - val_loss: 0.6993 - val_accuracy: 0.5249
Epoch 8/300

Epoch 59/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6600 - accuracy: 0.5878 - val_loss: 0.7109 - val_accuracy: 0.5028
Epoch 60/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6646 - accuracy: 0.6072 - val_loss: 0.7179 - val_accuracy: 0.5470
Epoch 61/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6559 - accuracy: 0.6030 - val_loss: 0.7166 - val_accuracy: 0.5083
Epoch 62/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6622 - accuracy: 0.6086 - val_loss: 0.7179 - val_accuracy: 0.5138
Epoch 63/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6579 - accuracy: 0.5947 - val_loss: 0.7195 - val_accuracy: 0.5193
Epoch 64/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6527 - accuracy: 0.6072 - val_loss: 0.7159 - val_accuracy: 0.5083
Epoch 65/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6553 - accuracy: 0.6100 - val_loss: 0.7191 - val_accuracy: 0.5083
Epoch 

37/37 [==============================] - 0s 7ms/step - loss: 0.6352 - accuracy: 0.6335 - val_loss: 0.7506 - val_accuracy: 0.4696
Epoch 117/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6360 - accuracy: 0.6307 - val_loss: 0.7636 - val_accuracy: 0.4586
Epoch 118/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6281 - accuracy: 0.6293 - val_loss: 0.7758 - val_accuracy: 0.4475
Epoch 119/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6226 - accuracy: 0.6238 - val_loss: 0.7731 - val_accuracy: 0.4530
Epoch 120/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6343 - accuracy: 0.6183 - val_loss: 0.7681 - val_accuracy: 0.4586
Epoch 121/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6273 - accuracy: 0.6349 - val_loss: 0.7778 - val_accuracy: 0.4365
Epoch 122/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6296 - accuracy: 0.6266 - val_loss: 0.7748 - val_accuracy: 0.4420
Epoch 123/300

37/37 [==============================] - 0s 6ms/step - loss: 0.6199 - accuracy: 0.6473 - val_loss: 0.8265 - val_accuracy: 0.4530
Epoch 174/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5998 - accuracy: 0.6556 - val_loss: 0.8582 - val_accuracy: 0.4586
Epoch 175/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6071 - accuracy: 0.6528 - val_loss: 0.8215 - val_accuracy: 0.4475
Epoch 176/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5932 - accuracy: 0.6639 - val_loss: 0.8573 - val_accuracy: 0.4586
Epoch 177/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5931 - accuracy: 0.6584 - val_loss: 0.8249 - val_accuracy: 0.4475
Epoch 178/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5911 - accuracy: 0.6556 - val_loss: 0.8619 - val_accuracy: 0.4420
Epoch 179/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6198 - accuracy: 0.6432 - val_loss: 0.8486 - val_accuracy: 0.4254
Epoch 180/300

37/37 [==============================] - 0s 7ms/step - loss: 0.5729 - accuracy: 0.6805 - val_loss: 0.9855 - val_accuracy: 0.4365
Epoch 231/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5717 - accuracy: 0.6680 - val_loss: 0.9486 - val_accuracy: 0.4365
Epoch 232/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5652 - accuracy: 0.6791 - val_loss: 0.9160 - val_accuracy: 0.4420
Epoch 233/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5796 - accuracy: 0.6846 - val_loss: 0.9392 - val_accuracy: 0.4586
Epoch 234/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5462 - accuracy: 0.6888 - val_loss: 0.9125 - val_accuracy: 0.4420
Epoch 235/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5727 - accuracy: 0.6805 - val_loss: 0.9455 - val_accuracy: 0.4530
Epoch 236/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5625 - accuracy: 0.6708 - val_loss: 0.9453 - val_accuracy: 0.4475
Epoch 237/300

37/37 [==============================] - 0s 7ms/step - loss: 0.5527 - accuracy: 0.6805 - val_loss: 1.0235 - val_accuracy: 0.4586
Epoch 288/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5352 - accuracy: 0.6916 - val_loss: 1.1238 - val_accuracy: 0.4254
Epoch 289/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5351 - accuracy: 0.6874 - val_loss: 1.1513 - val_accuracy: 0.4088
Epoch 290/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5323 - accuracy: 0.6888 - val_loss: 1.0715 - val_accuracy: 0.4696
Epoch 291/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5480 - accuracy: 0.6971 - val_loss: 1.0835 - val_accuracy: 0.4530
Epoch 292/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5350 - accuracy: 0.7012 - val_loss: 1.0641 - val_accuracy: 0.4254
Epoch 293/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5154 - accuracy: 0.7082 - val_loss: 1.0923 - val_accuracy: 0.4475
Epoch 294/300

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              904 non-null    int64  
 1   age                 904 non-null    int64  
 2   workout_experience  904 non-null    int64  
 3   workout_time        904 non-null    int64  
 4   weight              904 non-null    int64  
 5   height              904 non-null    int64  
 6   bmi                 904 non-null    float64
 7   fitness_goal        904 non-null    int64  
dtypes: float64(1), int64(7)
memory usage: 56.6 KB


In [28]:
input_data = {
    'gender' : 1,
    'age'    :21,
    'workout_experience' : 0,
    'workout_time' : 60,
    'weight' : 65,
    'height' : 170,
    'bmi' : 29
}

In [29]:
input_dict = {}

In [30]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [31]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([21])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([65])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>,
 'bmi': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([29])>}

In [32]:
predictions = model.predict(input_dict)

1/1 [==============================] - 1s 595ms/step


In [33]:
predictions

array([[0.96186364]], dtype=float32)

In [34]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !


In [35]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'model_1_v5.pkl')

KeyboardInterrupt: 